In [41]:
#create a function to pull in data from dropbox 
#import all necessary libraries

import pandas as pd
import numpy as np
from dotenv import load_dotenv
from dotenv import find_dotenv
import os
from pathlib import Path
import requests

import pathlib
import pandas as pd
import dropbox
from dropbox.exceptions import AuthError



In [21]:
#get .env variables

load_dotenv()


#set up the path to the .env file
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

#get api token from .env file
link = os.getenv("api_box")
key= os.getenv("ak")
sec=os.getenv("as")

In [66]:
#set up dropbox
dbx = dropbox.Dropbox(key)

#set up the path to the file in dropbox
path = '/test.csv'


In [67]:
#define a function using oauth2 to authenticate with dropbox
def dropbox_list_files(path):
    """Return a Pandas dataframe of files in a given Dropbox folder path in the Apps directory.
    """

    dbx = dropbox_connect()

    try:
        files = dbx.files_list_folder(path).entries
        files_list = []
        for file in files:
            if isinstance(file, dropbox.files.FileMetadata):
                metadata = {
                    'name': file.name,
                    'path_display': file.path_display,
                    'client_modified': file.client_modified,
                    'server_modified': file.server_modified
                }
                files_list.append(metadata)

        df = pd.DataFrame.from_records(files_list)
        return df.sort_values(by='server_modified', ascending=False)

    except Exception as e:
        print('Error getting list of files from Dropbox: ' + str(e))



In [68]:
dropbox_list_files('/Apps/OBDLink/CsvLogs/WVGEF9BP9GD001746')

Unable to refresh access token without                 refresh token and app key


Error getting list of files from Dropbox: AuthError('9666394b88634c53b68381cc3860308b', AuthError('expired_access_token', None))


In [34]:
#verify dropbox connection works

dbx_link.users_get_current_account()





Unable to refresh access token without                 refresh token and app key


AuthError: AuthError('5e16039c8f9f4501925ac7254cc24709', AuthError('expired_access_token', None))

In [4]:
'/Apps/OBDLink/CsvLogs/WVGEF9BP9GD001746'


In [131]:
#import csv from path
#drop first row and column
df1 = pd.read_csv('CSVLog_20221130_091051.csv', skiprows=1)


In [133]:
#drop first column
df1_col=df1.iloc[:,1:]
df=df1_col

In [134]:



#normalize the data for a convolutional neural network

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

#split the data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, df, test_size=0.2, random_state=42)

#reshape the data for the neural network
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

#build the model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

#fit the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

#make predictions
y_pred = model.predict(X_test)

#denormalize the data
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)

#calculate the mean squared error
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)






ModuleNotFoundError: No module named 'tensorflow'